In [1]:
import os
import osmnx as ox
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.types import LongType
from datetime import datetime

In [2]:
databricks = False
if not databricks:
    from util import folder
    data_dir = folder.DATA_DIR
    spark = SparkSession.builder.getOrCreate()
    clean_dbfs = (data_dir + "/FOIL/FOIL*/clean_data_*.gz.parquet")
    node_dbfs  = (data_dir + "/coordinate_to_node.gz.parquet")
else:
    data_dir = "/dbfs/mnt/group01"
    clean_dbfs = (data_dir + "/FOIL/FOIL*/clean_data_*.gz.parquet").replace("/dbfs", "")
    node_dbfs  = (data_dir + "/coordinate_to_node.gz.parquet").replace("/dbfs", "")

map_dir = os.path.join(data_dir, "nyc_map")

In [3]:
nyc_map = ox.load_graphml(os.path.join(map_dir, "NYC.mph"))

In [4]:
new_df = spark.read.parquet(clean_dbfs)\
    .groupBy(func.col("pickup_latitude"), func.col("pickup_longitude"))\
    .count().alias("count")
# new_df.show()

In [5]:
start = datetime.now()

def get_node(lat, lon):
    osmnx_id = ox.get_nearest_node(nyc_map, (lat,lon))
    return int(osmnx_id)

get_node_udf = func.udf(get_node, LongType())
p_df = new_df.withColumn('node_id', get_node_udf('pickup_latitude', 'pickup_longitude')).repartition(100, "node_id")
# p_df.show()
tlen = datetime.now()-start
print("time: {}".format(tlen))

time: 0:00:12.513110


In [6]:
p_df.write.mode("overwrite").option("compression", "gzip").parquet(node_dbfs)

Py4JJavaError: An error occurred while calling o55.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at org.apach